# Step17, 메모리 관리와 순환 참조 

성능을 개선할 수 있는 대책(기술)을 DeZero에 도입 

## 17.1 메모리 관리 
파이썬은 필요 없어진 객체를 메모리에서 자동으로 삭제한다.  
불필요한 객체는 파이썬 인터프리터가 제거해준다.  

그렇더라도 코드를 제대로 작성하지 않으면 **메모리 누수**(memory leak) 또는 **메모리 부족**(out of memory)등의 문제가 발생한다.  
특히 신경망에서는 큰 데이터를 다루는 경우가 많아서 메모리 관리를 제대로 하지 않으면 실행 시간이 오래 걸리는 일이 자주 발생한다.

파이썬의 메모리 관리  
파이썬(정확하게는 CPython)의 메모리 관리는 두가지 방식으로 진행된다.  
1. 참조(reference)수를 세는 방식 : '참조 카운트'  
2. 세대(generation)를 기준으로 쓸모없어진 객체(garbage)를 회수(collection)하는 방식 : 'GC'(Garbage Collection)



## 17.2 참조 카운트 방식의 메모리 관리 

파이썬 메모리 관리의 기본은 '참조 카운트'  
참조 카운트 : 구조가 간단하고 속도가 빠름  

- 모든 객체는 참조 카운트가 0인 상태로 생성  
- 다른 객체가 참조할 때마다 1씩 증가  
- 객체에 대한 참조가 끊길 때마다 1만큼 감소 --> 0이 되면 파이썬 인터프리터가 회수  

이런 방식으로 객체가 더 이상 필요없어지면 즉시 메모리에서 삭제 


참조 카운트가 증가하는 경우
- 대입 연산자를 사용할 때
- 함수에 인수로 전달할 때 
- 컨테이너 타입 객체(리스트,튜플,클래스)에 추가할 때 

개념을 설명하기 위한 수도 코드를 살펴보자

In [2]:
class obj:
    pass 
def f(x):
    print(x)

a = obj()   # 변수에 대입 : 참조 카운트 1 
f(a)        # 함수에 전달: 참조 카운트 2 
            # 함수 완료 : 참조 카운트 1 
a = None    # 대입 해제 : 참조 카운트 0


In [4]:
a = obj()   # a의 참조 카운트 : 1
b = obj()   # b의 참조 카운트 : 1
c = obj()   # c의 참조 카운트 : 1

a.b = b     # a가 b를 참조한다. b의 참조 카운트 : 2
b.c = c     # b가 c를 참조한다. c의 참조 카운트 : 2

a = b = c = None    # 해제되어 -1씩 

이처럼 참조 카운트 방식은 간단하다.  
하지만 '순환 참조'는 해결할 수 없다.

## 17.3 순환 참조 
아래는 순환참조(circular reference)를 설명하기 위한 수도코드이다

In [6]:
a = obj()   # 1
b = obj()   # 1
c = obj()   # 1

a.b = b     # 2
b.c = c     # 2
c.a = a     # 2

a=b=c=None  # 1

c에서 a로의 참조가 추가되었다. 그래서 3개의 객체가 원 모양을 이루며 서로가 서로를 참조하게 된다.  
이 상태가 바로 순환 참조이다.

마지막에 대입 해제를 하였지만 서로가 서로를 참조하기 때문에 참조 카운트는 모두 1로 남아있고 사용자는 이들 3 객체 중 어느 것에도 접근할 수 없다(즉, 모두 불필요한 객체이다.)  
a = b = c = None 을 실행하는 것으로는 순환 참조의 참조 카운트가 0이 되지않고, 결과적으로 메모리에서 삭제되지 않는다.  
때문에 다른 메모리 관리 방식이 필요하다.  
--> GC(generational garbage collection)

GC는 참조 카운트보다 열리한 방법으로 불필요한 객체를 찾아낸다.(이 책에서는 설명을 생략)  
GC는 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출된다. 물론 명시적으로 호출할수도 있다.(import gc, gc.collect())

메모리 해제를 GC에게 미루다 보면 프로그램의 전체 메모리 사용량이 커지는 원인이 된다.  
머신러닝, 신경망에서 메모리는 중요한 자원이다.  
따라서 순환 참조를 만들지 않는것이 좋다.

하지만 변수와 함수의 연결하는 방식에 순환 참조가 숨어 있다.

Function 인스턴스는 입력 Variable 과 출력 Variable을 참조한다.   
출력 Variable 인스턴스는 창조자인 Function 인스턴스를 참조한다.  

Function 인스턴스 <--> 출력 Variable 인스턴스 : 순환 참조 관계 

표준 파이썬 모듈인 weakref로 해결


## 17.4 weakref 모듈

파이썬에서는 weakref.ref 함수를 사용하여 약한 참조(weak reference)를 만들 수 있다.  
약한 참조 : 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능 

In [7]:
import weakref
import numpy as np 

a = np.array([1,2,3])
b = weakref.ref(a)

b

<weakref at 0x000001FB3E81E9A0; to 'numpy.ndarray' at 0x000001FB3E81E940>

In [8]:
b()

array([1, 2, 3])

a를 일반적인 방식으로 참조  
b는 약한 참조  
b를 출력해보면 ndarray를 가리키는 약한 참조(weakref)임을 확인할 수 있다.  
참조된 데이터에 접근하려면 b()하면된다.

이 weakref 구조를 DeZero에 도입한다.

In [10]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

위와 같이 인스턴스 변수 self.outputs가 대상을 약한 참조로 가리키게 변경한다.  
그 결과 '함수'는 '출력 변수'를 약하게 참조한다.  

또한 이 변경의 여파로 다른 클래스에서 Function클래스의 outputs를 참조하는 코드도 수정해야 한다. 

In [11]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            # gys = [output.grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.


## 17.5 동작 확인 



In [15]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)

In [16]:
for i in range(10):
    x = Variable(np.random.randn(10000))    # 거대한 데이터 
    y = square(square(square(x)))

for문이 두 번째 반복될 때 x와 y가 덮어 써진다. 그러면 사용자는 이전의 계산 그래프를 더 이상 참조하지 않게 된다.  
참조 카운트가 0이 되므로 이 시점에 계산 그래프에 사용된 메모리가 바로 삭제된다.

이것으로 DeZero 순환 참조 문제가 해소되었다.